## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/Weather_Database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Hermanus,-34.4187,19.2345,59.81,77,37,17.65,ZA,2022-07-28 23:07:20
1,1,Belm,52.3000,8.1333,64.56,59,100,9.62,DE,2022-07-28 23:07:20
2,2,Asfi,32.2994,-9.2372,70.29,88,0,15.39,MA,2022-07-28 23:07:21
3,3,Longyearbyen,78.2186,15.6401,40.84,93,75,9.22,SJ,2022-07-28 23:07:21
4,4,Praia,14.9215,-23.5087,79.50,75,2,11.99,CV,2022-07-28 23:03:08


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? -20
What is the maximum temperature you would like for your trip? 120


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Hermanus,-34.4187,19.2345,59.81,77,37,17.65,ZA,2022-07-28 23:07:20
1,1,Belm,52.3000,8.1333,64.56,59,100,9.62,DE,2022-07-28 23:07:20
2,2,Asfi,32.2994,-9.2372,70.29,88,0,15.39,MA,2022-07-28 23:07:21
3,3,Longyearbyen,78.2186,15.6401,40.84,93,75,9.22,SJ,2022-07-28 23:07:21
4,4,Praia,14.9215,-23.5087,79.50,75,2,11.99,CV,2022-07-28 23:03:08
5,5,Mao,39.8885,4.2658,77.56,78,0,4.61,ES,2022-07-28 23:07:22
6,6,Mahebourg,-20.4081,57.7000,70.25,77,5,10.29,MU,2022-07-28 23:07:22
7,7,Los Llanos De Aridane,28.6585,-17.9182,69.55,93,0,18.41,ES,2022-07-28 23:07:22
8,8,Sao Filipe,14.8961,-24.4956,79.88,74,13,13.58,CV,2022-07-28 23:07:23
9,9,Ouargaye,11.5033,0.0572,76.51,82,96,10.67,BF,2022-07-28 23:07:23


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID       667
City          667
Lat           667
Lng           667
Max Temp      667
Humidity      667
Cloudiness    667
Wind Speed    667
Country       660
Date          667
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
new_cities_df = preferred_cities_df.dropna()
new_cities_df.count()

City_ID       660
City          660
Lat           660
Lng           660
Max Temp      660
Humidity      660
Cloudiness    660
Wind Speed    660
Country       660
Date          660
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = new_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Hermanus,ZA,59.81,-34.4187,19.2345,
1,Belm,DE,64.56,52.3000,8.1333,
2,Asfi,MA,70.29,32.2994,-9.2372,
3,Longyearbyen,SJ,40.84,78.2186,15.6401,
4,Praia,CV,79.50,14.9215,-23.5087,
...,...,...,...,...,...,...
662,Pimenta Bueno,BR,79.70,-11.6725,-61.1936,
663,Kretinga,LT,53.35,55.8758,21.2508,
664,Saint Anthony,US,76.03,45.0205,-93.2180,
665,Jasper,US,89.65,37.2001,-94.3502,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [11]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()

City          660
Country       660
Max Temp      660
Lat           660
Lng           660
Hotel Name    660
dtype: int64

In [10]:
# 8a. Create the output File (CSV)

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 

# 11b. Display the figure
